In [1]:
import pandas as pd
import numpy as np


In [2]:
PATH ='/media/dat/dataset/YOUTUBE_BB/youtube_boundingboxes_detection_validation.csv'

In [3]:
def get_video_ids(csv_path=None, category='car', min_area=0.1, max_area=0.3, threshold=0.6):
    """
    Return a list of video ids that satisfies the condition as:
    
       * Contain at least one object of the specified category
       * All the object's area compared to the video size should be:
                * Less than the max_area and greater than min_area and
                * Percentage of satisfied area > threshold
    
    # This only works for Youtube BBoxes 
    # https://research.google.com/youtube-bb/download.html    
    """
    if csv_path==None: 
        raise IOError("Please specify path to Youtube bounding box file")
     
    df = pd.read_csv(csv_path,names=['youtube_id','timestamp','class_id','class_name',
                                     'object_id','object_presense',
                                     'xmin','xmax','ymin','ymax'])
    
    df = df[df['class_name'] == category]
    grouped = df.groupby('youtube_id')
           
    # Only get videos that have object are bigger than 30% of the frame
    def filter_obj(group, min_area=0.05, max_area=0.30, threshold=0.6):
        area = (group['xmax'] - group['xmin'])*(group['ymax'] - group['ymin'])
        area = (area > min_area) & (area  < max_area)
        if area.any() is False:
            return False
        else:    
            small_areas = area[area==True]
            is_good = (len(small_areas)/(1.0*len(area))) >= threshold
            return is_good
    
    # Filter grouped
    grouped = grouped.filter(lambda x: filter_obj(x, 
                                                  min_area=min_area, 
                                                  max_area=max_area, 
                                                  threshold=threshold)).groupby('youtube_id')
   
    return grouped

grouped = get_video_ids(PATH, category='car', min_area=0.1, max_area=0.5, threshold=0.7)
print(len(grouped))

159


In [ ]:
from IPython.display import YouTubeVideo

idx = np.random.randint(0, len(grouped.groups))
saved_id = list(grouped.groups.keys())[idx]
YouTubeVideo(list(grouped.groups.keys())[idx])

In [ ]:
print(grouped.groups)

In [ ]:
# Good id jDGrgBt83DU
YouTubeVideo('jDGrgBt83DU')